In [1]:
from pandas.io.json import json_normalize
from nltk.tokenize import WordPunctTokenizer
from rnnmorph.predictor import RNNMorphPredictor
from deeppavlov import configs, build_model
import pandas as pd
import string
import json
import random
import spacy
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

# Парсинг json

### SQuAD

In [3]:
def load_json(file):
    data = json.load(f)
    data_par = json_normalize(data=data['data'], record_path=['paragraphs'], meta=['title'])
    
    return data_par

In [3]:
f = open('squad/train-v2.0.json')
squad_train = load_json(f)
f.close()

In [128]:
f = open('squad/dev-v2.0.json')
squad_dev = load_json(f)
f.close()

In [5]:
def json_to_df(data_par):
    new_df = pd.DataFrame()
    for idx, listOfDicts in data_par[['qas']].itertuples():
        qas = pd.DataFrame.from_dict(listOfDicts)
        qas['title'] = data_par['title'][idx]
        qas['context'] = data_par['context'][idx]
        qas['answer'] = ''
        qas['answer_start'] = ''
        for ind, answer in qas[['answers']].itertuples():
            if answer == []:
                answer = qas['plausible_answers'][ind]
            try:
                dic = random.sample(answer, 1)[0]
            except ValueError:
                pass
            qas['answer'][ind] = dic.get('text')
            qas['answer_start'][ind] = dic.get('answer_start')
        new_df = new_df.append(qas)

    new_df = new_df.drop('answers', axis=1)
    new_df = new_df.drop('plausible_answers', axis=1)   
    new_df = new_df.drop('id', axis=1)
    new_df = new_df.reset_index()
    new_df = new_df.drop('index', axis=1)
    
    return new_df

In [ ]:
squad_train = json_to_df(squad_train)
print(len(squad_train))

In [7]:
squad_train.head(10)

,answer,answer_start,context,is_impossible,question,title
0,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,When did Beyonce start becoming popular?,Beyoncé
1,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,What areas did Beyonce compete in when she was...,Beyoncé
2,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,When did Beyonce leave Destiny's Child and bec...,Beyoncé
3,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,In what city and state did Beyonce grow up?,Beyoncé
4,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,In which decade did Beyonce become famous?,Beyoncé
5,Destiny's Child,320,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,In what R&B group was she the lead singer?,Beyoncé
6,Dangerously in Love,505,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,What album made her a worldwide known artist?,Beyoncé
7,Mathew Knowles,360,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,Who managed the Destiny's Child group?,Beyoncé
8,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,When did Beyoncé rise to fame?,Beyoncé
9,lead singer,290,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,False,What role did Beyoncé have in Destiny's Child?,Beyoncé


In [130]:
squad_dev = json_to_df(squad_dev)
print(len(squad_dev))

11873


In [109]:
squad_dev.head(10)

,question,is_impossible,title,context,answer,answer_start
0,In what country is Normandy located?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,France,159
1,When were the Normans in Normandy?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries,94
2,From which countries did the Norse originate?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway",256
3,Who was the Norse leader?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,Rollo,308
4,What century did the Normans first gain their ...,False,Normans,The Normans (Norman: Nourmands; French: Norman...,10th,671
5,Who gave their name to Normandy in the 1000's ...,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Normans,4
6,What is France a region of?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Normandy,137
7,Who did King Charles III swear fealty to?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Rollo,308
8,When did the Frankish identity emerge?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,10th century,671
9,Who was the duke in the battle of Hastings?,False,Normans,"The Norman dynasty had a major political, cult...",William the Conqueror,1022


### SberQuAD

In [4]:
f = open('sberquad/train-v1.1.json')
sberquad_train = load_json(f)
f.close()

In [5]:
f = open('sberquad/dev-v1.1.json')
sberquad_dev = load_json(f)
f.close()

In [6]:
def json_to_df_sber(data_par):
    new_df = pd.DataFrame({'context': data_par['context']})
    new_df['question'] = ''
    new_df['answer'] = ''
    new_df['answer_start'] = ''
    
    for idx, listOfDicts in data_par[['qas']].itertuples():
        dic = listOfDicts[0]
        new_df['question'][idx] = dic.get('question')
        answer = dic.get('answers')[0]
        new_df['answer'][idx] = answer.get('text')
        new_df['answer_start'][idx] = answer.get('answer_start')
    
    return new_df

In [7]:
sberquad_train = json_to_df_sber(sberquad_train)
print(len(sberquad_train))

45328


In [8]:
sberquad_train.head(10)

,context,question,answer,answer_start
0,В допетровское время искусство в России имело ...,Чем на Руси ограничивались проявления пластики?,"литьем небольших крестов, образов-складней, вы...",583
1,По данным рейтинга Top 100 самых филиальных ба...,Какое место занимает УБРиР по даным рейтинга T...,9 место,115
2,Смарт-стекло — современный собирательный терми...,Какие свойства в общем способно изменять смарт...,оптические,205
3,Во время интервенции в Россию Речи Посполитой ...,"Как звали воеводу Дорогобужа, который сдал гор...",Иван Ададуров,287
4,В начале 1823 года Грибоедов на время покинул ...,В каком году Грибоедов написал водевиль Кто бр...,1823,9
5,Так как некоторые из римских месяцев были назв...,Кто предполагал существование гипотетического ...,Якоб Гримм,535
6,Важным инструментом сохранения редких видов жи...,в какое приложение Бернской конвенции занесён ...,приложение II,183
7,Постоянное городское население города Сочи — 4...,Какое место по числу жителей занимал Сочи сред...,3-е место,408
8,"В 1980-х годах, с развитием панк-рока и появле...",Как звали гитариста группы конца 70-х Joy Divi...,Бернард Самнер,230
9,"Находясь в общем русле крымской истории, Феодо...",Насколько попытка контрнаступления второго Фед...,на несколько дней,416


In [9]:
sberquad_dev = json_to_df_sber(sberquad_dev)
print(len(sberquad_dev))

5036


In [10]:
sberquad_dev.head(10)

,context,question,answer,answer_start
0,"В Миссолонги Байрон заболел лихорадкой, продол...",Чем заболел Байрон в Миссолонги?,лихорадкой,28
1,От излишней потери тепла млекопитающих предохр...,Как отводятся излишки тепла у млекопитающих?,потоотделением,132
2,Хромосомные перестройки (хромосомные аберрации...,Что нарушают хромосомные аберрации?,Структуру хромосом.,74
3,Первый в мире городской автобус с двигателем в...,В каком городе первый в мире городской автобус...,в Лондоне,103
4,Фродом являются также и операции с поддельными...,В каком режиме проходят подлимитные операции?,в режиме офлайн,455
5,Слоговое письмо представляет собой набор письм...,Находит ли отражение в алфавите фонетическая с...,не находит,368
6,Весной 1890 года художник перебрался в Овер-сю...,Каким стал стиль последних работ художника?,нервным и гнетущим,232
7,В 2008 году родилось 1147 человек у граждан Шв...,Сколько человек родилось в 2008 году у граждан...,1147 человек,21
8,Иногда говорят о возможности провести GNU GPL ...,Какая статья и пункт дает возможность статья д...,пункт 2 статьи 434,558
9,Первым пособием по товароведению в России обыч...,Какими русскими купцами был накоплен значимый ...,купцами-новгородцами,245


# Очистка ответов от пунктуации

In [11]:
def clean_answer(answer):
    answer = answer.strip(string.punctuation)
    answer = answer.strip()

    return answer

In [12]:
sberquad_dev['answer'] = sberquad_dev['answer'].apply(lambda x: clean_answer(x))
sberquad_train['answer'] = sberquad_train['answer'].apply(lambda x: clean_answer(x))
squad_dev['answer'] = squad_dev['answer'].apply(lambda x: clean_answer(x))
squad_train['answer'] = squad_train['answer'].apply(lambda x: clean_answer(x))

# English

In [23]:
nlp = spacy.load("en_core_web_sm")

In [132]:
def features_en(text, answer, answer_start, nlp):
    
    right = text[0:answer_start]
    answer = text[answer_start:answer_start+len(answer)+1]
    left = text[answer_start+len(answer)+1:len(text)+1]    
    
    pos = []
    ner = []
    case = []
    bio = []
    tokenized = []
    
    right_doc = nlp(right)
    answer_doc = nlp(answer)
    left_doc = nlp(left)
    
    for token in right_doc:
        if token.text != '' and not token.text.isspace():
            tokenized.append(token.text.lower())
            pos.append(token.pos_)

            if token.ent_type_ == '':
                ner.append('O')
            else:
                ner.append(token.ent_type_)

            if token.text[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            bio.append('O')
    
    for token in answer_doc:
        if token.text != '' and not token.text.isspace():
            tokenized.append(token.text.lower())
            pos.append(token.pos_)

            if token.ent_type_ == '':
                ner.append('O')
            else:
                ner.append(token.ent_type_)

            if token.text[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            if token.i == 0:
                bio.append('B')
            else:
                bio.append('I')
    
    for token in left_doc:
        if token.text != '' and not token.text.isspace():
            tokenized.append(token.text.lower())
            pos.append(token.pos_)

            if token.ent_type_ == '':
                ner.append('O')
            else:
                ner.append(token.ent_type_)

            if token.text[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            bio.append('O')
                
    return (' '.join(pos)), (' '.join(ner)), (' '.join(case)), (' '.join(bio)), (' '.join(tokenized))

In [133]:
squad_dev['POS'] = ''
squad_dev['NER'] = ''
squad_dev['case'] = ''
squad_dev['BIO'] = ''
for idx, text, answer, answer_start in squad_dev[['context', 'answer', 'answer_start']].itertuples():
    squad_dev['POS'][idx], squad_dev['NER'][idx], squad_dev['case'][idx], squad_dev['BIO'][idx], squad_dev['context'][idx] = features_en(text, answer, answer_start, nlp)
    

In [136]:
def tokenize_en(text):
    doc = nlp(text)
    tokenized = [token.text.lower() for token in doc]
    
    return ' '.join(tokenized)

In [137]:
squad_dev['question'] = squad_dev['question'].apply(lambda x: tokenize_en(x))

In [140]:
squad_dev = squad_dev.drop(['answer', 'answer_start', 'title', 'is_impossible'], axis=1)
squad_dev.head()

,question,context,POS,NER,case,BIO
0,in what country is normandy located ?,the normans ( norman : nourmands ; french : no...,DET PROPN PUNCT PROPN PUNCT NOUN PUNCT PROPN P...,O NORP O PERSON O PERSON O NORP O O O LANGUAGE...,UP UP LOW UP LOW UP LOW UP LOW UP LOW UP LOW U...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,when were the normans in normandy ?,the normans ( norman : nourmands ; french : no...,DET PROPN PUNCT PROPN PUNCT NOUN PUNCT PROPN P...,O NORP O PERSON O PERSON O NORP O O O LANGUAGE...,UP UP LOW UP LOW UP LOW UP LOW UP LOW UP LOW U...,O O O O O O O O O O O O O O O O O O O B I I I ...
2,from which countries did the norse originate ?,the normans ( norman : nourmands ; french : no...,DET PROPN PUNCT PROPN PUNCT NOUN PUNCT PROPN P...,O NORP O PERSON O PERSON O NORP O O O LANGUAGE...,UP UP LOW UP LOW UP LOW UP LOW UP LOW UP LOW U...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,who was the norse leader ?,the normans ( norman : nourmands ; french : no...,DET PROPN PUNCT PROPN PUNCT NOUN PUNCT PROPN P...,O NORP O PERSON O PERSON O NORP O O O LANGUAGE...,UP UP LOW UP LOW UP LOW UP LOW UP LOW UP LOW U...,O O O O O O O O O O O O O O O O O O O O O O O ...
4,what century did the normans first gain their ...,the normans ( norman : nourmands ; french : no...,DET PROPN PUNCT PROPN PUNCT NOUN PUNCT PROPN P...,O NORP O PERSON O PERSON O NORP O O O LANGUAGE...,UP UP LOW UP LOW UP LOW UP LOW UP LOW UP LOW U...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [141]:
squad_dev.to_csv('dataset/squad_dev.csv', index=False)

In [142]:
squad_train['POS'] = ''
squad_train['NER'] = ''
squad_train['case'] = ''
squad_train['BIO'] = ''
for idx, text, answer, answer_start in squad_train[['context', 'answer', 'answer_start']].itertuples():
    squad_train['POS'][idx], squad_train['NER'][idx], squad_train['case'][idx], squad_train['BIO'][idx], squad_train['context'][idx] = features_en(text, answer, answer_start, nlp)

In [145]:
squad_train['question'] = squad_train['question'].apply(lambda x: tokenize_en(x))

In [146]:
squad_train = squad_train.drop(['answer', 'answer_start', 'title', 'is_impossible'], axis=1)
squad_train.head()

,context,question,POS,NER,case,BIO
0,beyoncé giselle knowles - carter ( /biːˈjɒnseɪ...,when did beyonce start becoming popular ?,PROPN PROPN PROPN PUNCT PROPN PUNCT ADJ NOUN P...,O O O O O O O O O O O O O O O DATE DATE DATE D...,UP UP UP LOW UP LOW LOW LOW LOW UP LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,beyoncé giselle knowles - carter ( /biːˈjɒnseɪ...,what areas did beyonce compete in when she was...,PROPN PROPN PROPN PUNCT PROPN PUNCT ADJ NOUN P...,O O O O O O O O O O O O O O O DATE DATE DATE D...,UP UP UP LOW UP LOW LOW LOW LOW UP LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...
2,beyoncé giselle knowles - carter ( /biːˈjɒnseɪ...,when did beyonce leave destiny 's child and be...,PROPN PROPN PROPN PUNCT PROPN PUNCT ADJ NOUN P...,O O O O O O O O O O O O O O O DATE DATE DATE D...,UP UP UP LOW UP LOW LOW LOW LOW UP LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,beyoncé giselle knowles - carter ( /biːˈjɒnseɪ...,in what city and state did beyonce grow up ?,PROPN PROPN PROPN PUNCT PROPN PUNCT ADJ NOUN P...,O O O O O O O O O O O O O O O DATE DATE DATE D...,UP UP UP LOW UP LOW LOW LOW LOW UP LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...
4,beyoncé giselle knowles - carter ( /biːˈjɒnseɪ...,in which decade did beyonce become famous ?,PROPN PROPN PROPN PUNCT PROPN PUNCT ADJ NOUN P...,O O O O O O O O O O O O O O O DATE DATE DATE D...,UP UP UP LOW UP LOW LOW LOW LOW UP LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [147]:
squad_train.to_csv('dataset/squad_train.csv', index=False)

# Russian

In [13]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2019-12-20 14:04:28.971 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
I1220 14:04:28.971092 140166193248064 download.py:117] Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2019-12-20 14:04:33.698 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
I1220 14:04:33.698825 140166193248064 download.py:117] Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /home/julia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/julia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
predictor = RNNMorphPredictor(language="ru")

In [41]:
def features_ru(text, predictor, ner_model, answer, answer_start):
    
    right = text[0:answer_start]
    answer = text[answer_start:answer_start+len(answer)]
    left = text[answer_start+len(answer):len(text)+1]    
    
    pos = []
    ner = []
    case = []
    bio = []
    tokenized = []

    right_tokenized = WordPunctTokenizer().tokenize(right)
    answer_tokenized = WordPunctTokenizer().tokenize(answer)
    left_tokenized = WordPunctTokenizer().tokenize(left)
    
    for token in right_tokenized:
        if token != '' and not token.isspace():
            tokenized.append(token.lower())

            if token[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            bio.append('O')
    
    for token in answer_tokenized:
        if token != '' and not token.isspace():
            tokenized.append(token.lower())

            if token[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            if answer_tokenized.index(token) == 0:
                bio.append('B')
            else:
                bio.append('I')
    
    for token in left_tokenized:
        if token != '' and not token.isspace():
            tokenized.append(token.lower())

            if token[0].isupper():
                case.append('UP')
            else:
                case.append('LOW')
            bio.append('O')
    
    forms = predictor.predict(tokenized)
    pos = [form.pos for form in forms]
    ner = ner_model([tokenized])
        
    return (' '.join(pos)), (' '.join(ner[1][0])), (' '.join(case)), (' '.join(bio)), (' '.join(tokenized))
    

In [42]:
sberquad_dev['POS'] = ''
sberquad_dev['NER'] = ''
sberquad_dev['case'] = ''
sberquad_dev['BIO'] = ''
for idx, text, answer, answer_start in sberquad_dev[['context', 'answer', 'answer_start']].itertuples():
    try:
        sberquad_dev['POS'][idx], sberquad_dev['NER'][idx], sberquad_dev['case'][idx], sberquad_dev['BIO'][idx], sberquad_dev['context'][idx] = features_ru(text, predictor, ner_model, answer, answer_start)
    except RuntimeError:
        sberquad_dev = sberquad_dev.drop([idx], axis=0)

In [43]:
sberquad_dev['question'] = sberquad_dev['question'].apply(lambda x: ' '.join(WordPunctTokenizer().tokenize(x.lower())))

In [44]:
sberquad_dev = sberquad_dev.drop(['answer', 'answer_start'], axis=1)
sberquad_dev.head()

,context,question,POS,NER,case,BIO
0,"в миссолонги байрон заболел лихорадкой , продо...",чем заболел байрон в миссолонги ?,ADP NOUN NOUN VERB NOUN PUNCT VERB VERB DET DE...,O O O O O O O O O O O O O O O O O B-DATE I-DAT...,UP UP UP LOW LOW LOW LOW LOW LOW LOW LOW LOW L...,O O O O B O O O O O O O O O O O O O O O O O O ...
1,от излишней потери тепла млекопитающих предохр...,как отводятся излишки тепла у млекопитающих ?,ADP ADJ NOUN NOUN NOUN VERB ADJ NOUN PUNCT CON...,O O O O O O O O O O O O O O O O O O O O O O O ...,UP LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O B O ...
2,хромосомные перестройки ( хромосомные аберраци...,что нарушают хромосомные аберрации ?,ADJ NOUN PUNCT ADJ NOUN PUNCT PUNCT PART NOUN ...,O O O O O O O O O O O O O O O O O O O O O O O ...,UP LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW...,O O O O O O O O O O O B I O O O O O O O O O O ...
3,первый в мире городской автобус с двигателем в...,в каком городе первый в мире городской автобус...,ADJ ADP NOUN ADJ NOUN ADP NOUN ADJ NOUN VERB A...,B-ORDINAL O O O O O O O O O O O B-DATE I-DATE ...,UP LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW...,O O O O O O O O O O O O O O O O B I O O O O O ...
4,фродом являются также и операции с поддельными...,в каком режиме проходят подлимитные операции ?,NOUN VERB ADV PART NOUN ADP ADJ NOUN PUNCT VER...,O O O O O O O O O O O O O O O O O O O O O O O ...,UP LOW LOW LOW LOW LOW LOW LOW LOW UP LOW LOW ...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [45]:
sberquad_dev.to_csv('dataset/sberquad_dev.csv', index=False)

In [48]:
sberquad_train['POS'] = ''
sberquad_train['NER'] = ''
sberquad_train['case'] = ''
sberquad_train['BIO'] = ''
for idx, text, answer, answer_start in sberquad_train[['context', 'answer', 'answer_start']].itertuples():
    try:
        sberquad_train['POS'][idx], sberquad_train['NER'][idx], sberquad_train['case'][idx], sberquad_train['BIO'][idx], sberquad_train['context'][idx] = features_ru(text, predictor, ner_model, answer, answer_start)
    except RuntimeError:
        sberquad_train = sberquad_train.drop([idx], axis=0)

In [49]:
sberquad_train['question'] = sberquad_train['question'].apply(lambda x: ' '.join(WordPunctTokenizer().tokenize(x.lower())))

In [50]:
sberquad_train = sberquad_train.drop(['answer', 'answer_start'], axis=1)
sberquad_train.head()

,context,question,POS,NER,case,BIO
0,в допетровское время искусство в россии имело ...,чем на руси ограничивались проявления пластики ?,ADP ADJ NOUN NOUN ADP NOUN VERB DET NOUN VERB ...,O O O O O O O O O O O O O O O O O O O O O O O ...,UP LOW LOW LOW LOW UP LOW LOW LOW LOW LOW LOW ...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,по данным рейтинга top 100 самых филиальных ба...,какое место занимает убрир по даным рейтинга t...,ADP NOUN NOUN PUNCT NUM ADJ ADJ NOUN NOUN NOUN...,O O O O B-CARDINAL O O O B-GPE O B-ORG O O O O...,UP LOW LOW UP LOW LOW LOW LOW UP LOW UP LOW UP...,O O O O O O O O O O O O O O O O O O O O B I O ...
2,смарт - стекло — современный собирательный тер...,какие свойства в общем способно изменять смарт...,NOUN PUNCT NOUN PUNCT ADJ ADJ NOUN ADP NOUN PU...,O O O O O O O O O O O O O O O O O O O O O O O ...,UP LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW LOW...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,во время интервенции в россию речи посполитой ...,"как звали воеводу дорогобужа , который сдал го...",ADP NOUN NOUN ADP NOUN NOUN ADJ NOUN NUM PUNCT...,O O O O O O O B-DATE I-DATE I-DATE I-DATE I-DA...,UP LOW LOW LOW UP UP UP LOW LOW LOW LOW LOW LO...,O O O O O O O O O O O O O O O O O O O O O O O ...
4,в начале 1823 года грибоедов на время покинул ...,в каком году грибоедов написал водевиль кто бр...,ADP NOUN NUM NOUN NOUN ADP NOUN VERB NOUN CONJ...,O B-DATE I-DATE I-DATE B-PERSON O O O O O O O ...,UP LOW LOW LOW UP LOW LOW LOW LOW LOW LOW LOW ...,O O B O O O O O O O O O O O O O O O O O O O O ...


In [51]:
sberquad_train.to_csv('dataset/sberquad_train.csv', index=False)

In [52]:
len(sberquad_train)

45018